In [38]:
import pandas as pd
import numpy as np
from fancyimpute import MICE
%pylab inline

df = pd.read_csv('/home/lara/Documents/Repository/Capstone-1_WorldBank_GenderData/finalvars.csv')

# Supervised Learning Modules
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


Populating the interactive namespace from numpy and matplotlib


In [26]:
df.head(2)

,Unnamed: 0,country,life,bc,matdeath,teen,gdp,healthspend,eduspend
0,0,Afghanistan,55.125878,4.9,1100.0,153.8456,NaN,NaN,NaN
1,1,Afghanistan,55.487537,NaN,1050.0,150.0468,119.899037,NaN,NaN


In [36]:
#Select only rows that have a value for Contraceptive Use
df= df[np.isfinite(df.bc)]
df.head()

,Unnamed: 0,country,life,bc,matdeath,teen,gdp,healthspend,eduspend
0,0,Afghanistan,55.125878,4.9,1100.0,153.8456,NaN,NaN,NaN
3,3,Afghanistan,56.235293,10.3,941.0,140.4764,203.651041,0.607900,NaN
5,5,Afghanistan,57.027244,13.6,821.0,128.9332,257.175795,0.801919,NaN
6,6,Afghanistan,57.432561,18.6,776.0,123.1616,280.245644,1.062420,NaN
8,8,Afghanistan,58.225024,22.8,676.0,111.4708,384.131681,1.385584,NaN


In [43]:
#Eliminate 'unnamed' and 'country' columns
df = df[['life', 'bc', 'matdeath', 'teen', 'gdp', 'healthspend', 'eduspend']]

#use MICE algorithm in Fancy Impute package to interpolate remaining NaN values
micefilled = MICE().complete(df)
#turn micefilled nmpy array into pandas df
data = pd.DataFrame(micefilled, columns= df.columns)

[MICE] Completing matrix with shape (3945, 7)
[MICE] Starting imputation round 1/110, elapsed time 0.001
[MICE] Starting imputation round 2/110, elapsed time 0.020
[MICE] Starting imputation round 3/110, elapsed time 0.032
[MICE] Starting imputation round 4/110, elapsed time 0.044
[MICE] Starting imputation round 5/110, elapsed time 0.086
[MICE] Starting imputation round 6/110, elapsed time 0.104
[MICE] Starting imputation round 7/110, elapsed time 0.138
[MICE] Starting imputation round 8/110, elapsed time 0.153
[MICE] Starting imputation round 9/110, elapsed time 0.187
[MICE] Starting imputation round 10/110, elapsed time 0.195
[MICE] Starting imputation round 11/110, elapsed time 0.200
[MICE] Starting imputation round 12/110, elapsed time 0.230
[MICE] Starting imputation round 13/110, elapsed time 0.270
[MICE] Starting imputation round 14/110, elapsed time 0.293
[MICE] Starting imputation round 15/110, elapsed time 0.301
[MICE] Starting imputation round 16/110, elapsed time 0.307
[MI

In [44]:
data.head()

,life,bc,matdeath,teen,gdp,healthspend,eduspend
0,55.125878,4.900000,1100.0,153.8456,-15633.908148,2.167493,4.406384
1,55.487537,31.304324,1050.0,150.0468,119.899037,2.948343,4.420182
2,55.857195,31.692872,996.0,146.2480,192.153528,0.550723,4.221319
3,56.235293,10.300000,941.0,140.4764,203.651041,0.607900,4.166311
4,56.626317,35.682468,881.0,134.7048,224.914712,0.969459,4.505312


In [45]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RF', RandomForestClassifier(n_jobs = -1, n_estimators = 500)))

In [46]:
Y = df.bc
X = pd.concat(
    [df[['gdp', 'healthspend', 'eduspend']]],
    axis = 1)

In [50]:
#a function to evaluate each model
def run_models(train_data):
    results = []
    names = []

    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state=11)
        cv_results = model_selection.cross_val_score(model, train_data, Y, cv=kfold, scoring='accuracy')
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
'''
    # boxplot algorithm comparison
    fig = pyplot.figure()
    fig.suptitle(title)
    ax = fig.add_subplot(111)
    pyplot.boxplot(results)
    ax.set_xticklabels(names)
    pyplot.ylim(0,1)
    pyplot.show()
    '''

'\n    # boxplot algorithm comparison\n    fig = pyplot.figure()\n    fig.suptitle(title)\n    ax = fig.add_subplot(111)\n    pyplot.boxplot(results)\n    ax.set_xticklabels(names)\n    pyplot.ylim(0,1)\n    pyplot.show()\n    '

In [51]:
run_models(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').